In [5]:

#coding:utf-8
from __future__ import print_function
 
import os
import random
import glob
import xml.etree.ElementTree as ET
 
def xml_reader(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    size = tree.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text),
                              int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text),
                              int(bbox.find('ymax').text)]
        objects.append(obj_struct)
    return width, height, objects
 
 
def voc2yolo(filename):
    classes_dict = {}
    with open("result/classes.names") as f:
        for idx, line in enumerate(f.readlines()):
            class_name = line.strip()
            classes_dict[class_name] = idx
    
    width, height, objects = xml_reader(filename)
 
    lines = []
    for obj in objects:
        x, y, x2, y2 = obj['bbox']
        class_name = obj['name']
        label = classes_dict[class_name]
        cx = (x2+x)*0.5 / width
        cy = (y2+y)*0.5 / height
        w = (x2-x)*1. / width
        h = (y2-y)*1. / height
        line = "%s %.6f %.6f %.6f %.6f\n" % (label, cx, cy, w, h)
        lines.append(line)
    print(filename)
    txt_name = filename.replace(".xml", ".txt").replace("result/Annotations", "/result/images")
    with open(txt_name, "w") as f:
        f.writelines(lines)
 
 
def get_image_list(image_dir, suffix=['jpg', 'jpeg', 'JPG', 'JPEG','png']):
    '''get all image path ends with suffix'''
    if not os.path.exists(image_dir):
        print("PATH:%s not exists" % image_dir)
        return []
    imglist = []
    for root, sdirs, files in os.walk(image_dir):
        if not files:
            continue
        for filename in files:
            filepath = r"C:\Users\simon\Desktop\cocoapi-master\PythonAPI\test" + os.path.join(root, filename) + "\n"
            if filename.split('.')[-1] in suffix:
                imglist.append(filepath)
    return imglist
 
 
def imglist2file(imglist):
    random.shuffle(imglist)
    train_list = imglist[:-tp]
    valid_list = imglist[-tp:]
    with open("train.txt", "w") as f:
        f.writelines(train_list)
    with open("valid.txt", "w") as f:
        f.writelines(valid_list)
 
 
if __name__ == "__main__":
    xml_path_list = glob.glob("result/Annotations/*.xml")
    count=0
    for xml_path in xml_path_list:
        voc2yolo(xml_path)
        count+=1
        
    train_percent = 0.9
    tp=int(count*train_percent)
    tp=count-tp
    imglist = get_image_list("result/images")
    imglist2file(imglist)

result/Annotations\000000000074.xml


FileNotFoundError: [Errno 2] No such file or directory: '/result/images\\000000000074.txt'